In [12]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)
print("done")

done


In [13]:
print("reading file")
X = pd.read_csv("jester-data-1.csv", delimiter=",", header=None) 
print(X.shape)

reading file
(24983, 101)


In [14]:
print(X.head())

   0      1      2      3      4     5     6     7     8      9    ...    91   \
0   74  -7.82   8.79  -9.66  -8.16 -7.52 -8.50 -9.85  4.17  -8.98  ...   2.82   
1  100   4.08  -0.29   6.36   4.37 -2.38 -9.66 -0.73 -5.34   8.88  ...   2.82   
2   49  99.00  99.00  99.00  99.00  9.03  9.27  9.03  9.27  99.00  ...  99.00   
3   48  99.00   8.35  99.00  99.00  1.80  8.16 -2.82  6.21  99.00  ...  99.00   
4   91   8.50   4.61  -4.17  -5.39  1.36  1.60  7.04  4.61  -0.44  ...   5.19   

     92     93     94     95     96     97     98     99     100  
0  99.00  99.00  99.00  99.00  99.00  -5.63  99.00  99.00  99.00  
1  -4.95  -0.29   7.86  -0.19  -2.14   3.06   0.34  -4.32   1.07  
2  99.00  99.00   9.08  99.00  99.00  99.00  99.00  99.00  99.00  
3  99.00  99.00   0.53  99.00  99.00  99.00  99.00  99.00  99.00  
4   5.58   4.27   5.19   5.73   1.55   3.11   6.55   1.80   1.60  

[5 rows x 101 columns]


In [20]:
print("drop")
X = X[1:-1]
print(X.head())

drop
     1      2      3      4     5     6     7     8      9      10   ...  \
2  99.00  99.00  99.00  99.00  9.03  9.27  9.03  9.27  99.00  99.00  ...   
3  99.00   8.35  99.00  99.00  1.80  8.16 -2.82  6.21  99.00   1.84  ...   
4   8.50   4.61  -4.17  -5.39  1.36  1.60  7.04  4.61  -0.44   5.73  ...   
5  -6.17  -3.54   0.44  -8.50 -7.09 -4.32 -8.69 -0.87  -6.65  -1.80  ...   
6  99.00  99.00  99.00  99.00  8.59 -9.85  7.72  8.79  99.00  99.00  ...   

     91     92     93     94     95     96     97     98     99     100  
2  99.00  99.00  99.00   9.08  99.00  99.00  99.00  99.00  99.00  99.00  
3  99.00  99.00  99.00   0.53  99.00  99.00  99.00  99.00  99.00  99.00  
4   5.19   5.58   4.27   5.19   5.73   1.55   3.11   6.55   1.80   1.60  
5  -3.54  -6.89  -0.68  -2.96  -2.18  -3.35   0.05  -9.08  -5.05  -3.45  
6  99.00  99.00  99.00  99.00  99.00   2.33  99.00  99.00  99.00  99.00  

[5 rows x 100 columns]


In [26]:
data = X.copy().values
ten_percent = (int)(data.shape[0] * 0.1)
data[-ten_percent:] = 99
print(data)

[[ 99.    99.    99.   ...,  99.    99.    99.  ]
 [ 99.     8.35  99.   ...,  99.    99.    99.  ]
 [  8.5    4.61  -4.17 ...,   6.55   1.8    1.6 ]
 ..., 
 [ 99.    99.    99.   ...,  99.    99.    99.  ]
 [ 99.    99.    99.   ...,  99.    99.    99.  ]
 [ 99.    99.    99.   ...,  99.    99.    99.  ]]


In [45]:

n_features = 10

latent_user_preferences = np.random.random((data.shape[0], n_features))
latent_jokes_features = np.random.random((data.shape[1], n_features))

print(latent_user_preferences.shape)
print(latent_jokes_features.shape)

(24979, 10)
(100, 10)


In [29]:
def predict_rating(user_id, joke_id):
    """ Predict a rating given a user_id and a joke_id.
    """
    user_preference = latent_user_preferences[user_id]
    joke_feature = latent_jokes_features[joke_id]
    
    return user_preference.dot(joke_feature)

def train(user_id, joke_id, rating, alpha = 0.0001):
    
    prediction_rating = predict_rating(user_id, joke_id)
    err =  (prediction_rating- rating);
    
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_jokes_features[joke_id]
    latent_jokes_features[joke_id] -= alpha * err * user_pref_values
    
    return err
    

def sgd(iterations = 300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0, iterations):
        error = []
        for user_id in range(0, latent_user_preferences.shape[0]):
            for joke_id in range(0, latent_jokes_features.shape[0]):
                rating = data[user_id][joke_id]
                if(rating != 99):
                    err = train(user_id, joke_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        print(mse)

In [31]:
print("training...")
sgd(5)
print("...done")

training...
24.9023092607
24.7132883557
24.6529126871
24.5947368904
24.5375656539
...done


In [32]:
predictions = latent_user_preferences.dot(latent_jokes_features.T)
print(predictions)

[[ 1.079  0.07   0.301 ...,  0.927 -0.129  1.773]
 [ 0.784  0.147  0.358 ...,  0.646  0.098  1.319]
 [ 0.828  0.175  0.47  ...,  0.901 -0.253  1.443]
 ..., 
 [ 0.759  0.096  0.496 ...,  0.763 -0.228  1.705]
 [ 1.086  0.145  0.511 ...,  0.96  -0.049  1.846]
 [ 0.906  0.113  0.392 ...,  0.818  0.015  1.363]]


In [36]:
values = [zip(data[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = X.columns
comparison_data.applymap(lambda (x,y): "(%2.3f|%2.3f)"%(x,y))

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,(99.000|1.079),(99.000|0.070),(99.000|0.301),(99.000|-1.767),(9.030|0.567),(9.270|1.860),(9.030|-0.462),(9.270|-0.550),(99.000|-0.998),(99.000|1.325),...,(99.000|2.326),(99.000|1.330),(99.000|2.500),(9.080|1.314),(99.000|1.158),(99.000|1.253),(99.000|1.874),(99.000|0.927),(99.000|-0.129),(99.000|1.773)
1,(99.000|0.784),(8.350|0.147),(99.000|0.358),(99.000|-1.191),(1.800|0.582),(8.160|1.210),(-2.820|-0.403),(6.210|-0.430),(99.000|-0.118),(1.840|1.392),...,(99.000|1.682),(99.000|0.971),(99.000|2.488),(0.530|1.133),(99.000|0.817),(99.000|1.386),(99.000|1.297),(99.000|0.646),(99.000|0.098),(99.000|1.319)
2,(8.500|0.828),(4.610|0.175),(-4.170|0.470),(-5.390|-1.423),(1.360|0.474),(1.600|1.578),(7.040|-0.206),(4.610|-0.671),(-0.440|-0.783),(5.730|1.059),...,(5.190|2.072),(5.580|1.183),(4.270|2.253),(5.190|1.327),(5.730|0.887),(1.550|1.371),(3.110|1.579),(6.550|0.901),(1.800|-0.253),(1.600|1.443)
3,(-6.170|0.766),(-3.540|0.014),(0.440|0.353),(-8.500|-1.431),(-7.090|0.496),(-4.320|1.360),(-8.690|-0.214),(-0.870|-0.581),(-6.650|-0.619),(-1.800|1.146),...,(-3.540|1.911),(-6.890|0.913),(-0.680|2.058),(-2.960|1.103),(-2.180|0.806),(-3.350|1.336),(0.050|1.573),(-9.080|0.772),(-5.050|0.135),(-3.450|1.277)
4,(99.000|1.120),(99.000|0.021),(99.000|0.388),(99.000|-1.553),(8.590|0.411),(-9.850|1.514),(7.720|-0.413),(8.790|-0.688),(99.000|-0.769),(99.000|1.510),...,(99.000|1.883),(99.000|1.062),(99.000|2.545),(99.000|1.306),(99.000|0.810),(2.330|1.342),(99.000|1.745),(99.000|0.893),(99.000|0.015),(99.000|1.594)
5,(6.840|0.488),(3.160|-0.021),(9.170|0.320),(-6.210|-1.024),(-8.160|0.402),(-1.700|0.998),(9.270|-0.278),(1.410|-0.421),(-5.190|-0.339),(-4.420|1.096),...,(7.230|1.605),(-1.120|0.563),(-0.100|1.891),(-5.680|0.830),(-3.160|0.610),(-3.350|0.765),(2.140|1.317),(-0.050|0.577),(1.310|0.243),(0.000|0.892)
6,(-3.790|0.832),(-3.540|0.260),(-9.420|0.224),(-6.890|-1.135),(-8.740|0.355),(-0.290|1.626),(-5.290|-0.243),(-8.930|-0.222),(-7.860|-0.877),(-1.600|0.859),...,(4.370|1.812),(-0.290|1.425),(4.170|1.930),(-0.290|0.852),(-0.290|1.067),(-0.290|1.133),(-0.290|1.591),(-0.290|0.566),(-3.400|-0.063),(-4.950|1.273)
7,(3.010|0.624),(5.150|0.340),(5.150|0.247),(3.010|-1.127),(6.410|0.447),(5.150|1.549),(8.930|-0.424),(2.520|-0.413),(3.010|-0.684),(8.160|0.935),...,(99.000|2.013),(4.470|1.171),(99.000|2.194),(99.000|0.839),(99.000|1.128),(99.000|1.121),(99.000|1.445),(99.000|0.608),(99.000|-0.194),(99.000|1.351)
8,(-2.910|0.929),(4.080|0.163),(99.000|0.486),(99.000|-1.273),(-5.730|0.548),(99.000|1.266),(2.480|-0.250),(-5.290|-0.586),(99.000|-0.576),(1.460|1.255),...,(99.000|1.603),(6.170|1.131),(99.000|2.507),(99.000|1.041),(99.000|0.678),(99.000|1.308),(99.000|1.327),(99.000|0.648),(99.000|-0.097),(99.000|1.766)
9,(1.310|1.336),(1.800|0.176),(2.570|0.478),(-2.380|-1.836),(0.730|0.393),(0.730|2.032),(-0.970|-0.400),(5.000|-0.886),(-7.230|-1.064),(-1.360|1.570),...,(1.460|2.374),(1.700|1.502),(0.290|2.877),(-3.300|1.728),(3.450|1.106),(5.440|1.902),(4.080|2.127),(2.480|1.177),(4.510|-0.229),(4.660|1.783)


In [54]:
data = X.copy().values
index = data.shape[0] - valid_count
error = []
print(index)
print(data.shape)

22482
(24979, 100)


In [55]:
print("training")
for user_id in range(index, latent_user_preferences.shape[0]):
    for joke_id in range(0, latent_jokes_features.shape[0]):
        rating = data[user_id][joke_id]
        if(rating != 99):
            pred = predictions[user_id][joke_id]
            error.append(pred - rating)
mse = (np.array(error) ** 2).mean()
print(mse)

training
26.6818556676
